In [1]:
#pip install pythae

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\arnab\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import pandas as pd
import sdv
import seaborn as sns
import matplotlib.pyplot as plt

#libraries needed for pythae
from pythae.models import VAEGAN, VAEGANConfig
from pythae.trainers import CoupledOptimizerAdversarialTrainerConfig
from pythae.pipelines.training import TrainingPipeline

#pytorch
import torch


device = "cuda" if torch.cuda.is_available() else "cpu"


In [2]:
real_data1=pd.read_csv('german_credit_data.csv')

In [3]:
real_data1.columns

Index(['checking_status', 'duration', 'credit_history', 'purpose',
       'credit_amount', 'savings_status', 'employment',
       'installment_commitment', 'personal_status', 'other_parties',
       'residence_since', 'property_magnitude', 'age', 'other_payment_plans',
       'housing', 'existing_credits', 'job', 'num_dependents', 'own_telephone',
       'foreign_worker', 'class'],
      dtype='object')

In [4]:
real_data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   checking_status         1000 non-null   object
 1   duration                1000 non-null   int64 
 2   credit_history          1000 non-null   object
 3   purpose                 1000 non-null   object
 4   credit_amount           1000 non-null   int64 
 5   savings_status          1000 non-null   object
 6   employment              1000 non-null   object
 7   installment_commitment  1000 non-null   int64 
 8   personal_status         1000 non-null   object
 9   other_parties           1000 non-null   object
 10  residence_since         1000 non-null   int64 
 11  property_magnitude      1000 non-null   object
 12  age                     1000 non-null   int64 
 13  other_payment_plans     1000 non-null   object
 14  housing                 1000 non-null   object
 15  exist

# Normalizing data

In [5]:
from rdt import HyperTransformer
ht = HyperTransformer()
ht.detect_initial_config(data=real_data1)
config=ht.get_config()
ht.fit(real_data1)
real_data=ht.transform(real_data1)

In [6]:
real_data

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,0.5310,6.0,0.6765,0.1400,1169.0,0.6945,0.4655,4.0,0.274,0.4535,...,0.473,67.0,0.407,0.3565,2.0,0.315,1.0,0.798,0.4815,0.35
1,0.8025,48.0,0.2650,0.1400,5951.0,0.3015,0.1695,2.0,0.703,0.4535,...,0.473,22.0,0.407,0.3565,1.0,0.315,1.0,0.298,0.4815,0.85
2,0.1970,12.0,0.6765,0.9200,2096.0,0.3015,0.6790,2.0,0.274,0.4535,...,0.473,49.0,0.407,0.3565,1.0,0.730,2.0,0.298,0.4815,0.35
3,0.5310,42.0,0.2650,0.6045,7882.0,0.3015,0.6790,2.0,0.274,0.9330,...,0.730,45.0,0.407,0.9460,1.0,0.315,2.0,0.298,0.4815,0.35
4,0.5310,24.0,0.8670,0.3970,4870.0,0.3015,0.1695,3.0,0.274,0.4535,...,0.923,53.0,0.407,0.9460,2.0,0.315,2.0,0.298,0.4815,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.1970,12.0,0.2650,0.6045,1736.0,0.3015,0.6790,3.0,0.703,0.4535,...,0.473,31.0,0.407,0.3565,1.0,0.730,1.0,0.298,0.4815,0.35
996,0.5310,30.0,0.2650,0.7465,3857.0,0.3015,0.1695,4.0,0.975,0.4535,...,0.730,40.0,0.407,0.3565,1.0,0.904,1.0,0.798,0.4815,0.35
997,0.1970,12.0,0.2650,0.1400,804.0,0.3015,0.4655,4.0,0.274,0.4535,...,0.166,38.0,0.407,0.3565,1.0,0.315,1.0,0.298,0.4815,0.35
998,0.5310,45.0,0.2650,0.1400,1845.0,0.3015,0.1695,4.0,0.274,0.4535,...,0.923,23.0,0.407,0.9460,1.0,0.315,1.0,0.798,0.4815,0.85


In [30]:
dum_col=['checking_status', 'credit_history', 'purpose',
        'savings_status', 'employment',
        'personal_status', 'other_parties',
        'property_magnitude',  'other_payment_plans',
       'housing', 'existing_credits', 'job', 'own_telephone',
       'foreign_worker']

In [31]:
#converting categorical columns to numerical using one-hot encoding
real_data = pd.get_dummies(real_data1, columns=dum_col, drop_first=True)

In [32]:
real_data['class']=real_data['class'].replace(['good'],'0')
real_data['class']=real_data['class'].replace(['bad'],'1')


In [33]:
real_data[['class']] = real_data[[ 'class']].apply(pd.to_numeric)

In [34]:
rea_data_csv=real_data.to_csv('real_data.csv', index = True)

In [35]:
real_data

,duration,credit_amount,installment_commitment,residence_since,age,num_dependents,class,checking_status_<0,checking_status_>=200,checking_status_no checking,...,housing_own,housing_rent,existing_credits_2,existing_credits_3,existing_credits_4,job_skilled,job_unemp/unskilled non res,job_unskilled resident,own_telephone_yes,foreign_worker_yes
0,6,1169,4,4,67,1,0,1,0,0,...,1,0,1,0,0,1,0,0,1,1
1,48,5951,2,2,22,1,1,0,0,0,...,1,0,0,0,0,1,0,0,0,1
2,12,2096,2,3,49,2,0,0,0,1,...,1,0,0,0,0,0,0,1,0,1
3,42,7882,2,4,45,2,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1
4,24,4870,3,4,53,2,1,1,0,0,...,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,12,1736,3,4,31,1,0,0,0,1,...,1,0,0,0,0,0,0,1,0,1
996,30,3857,4,4,40,1,0,1,0,0,...,1,0,0,0,0,0,0,0,1,1
997,12,804,4,4,38,1,0,0,0,1,...,1,0,0,0,0,1,0,0,0,1
998,45,1845,4,4,23,1,1,1,0,0,...,0,0,0,0,0,1,0,0,1,1


In [36]:
tr

NameError: name 'tr' is not defined

In [7]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

## pythae (vae) 

In [8]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader, random_split

class MyDataset(Dataset):
    def __init__(self,data):
        # Load data from CSV file using pandas
        df = real_data
        # Convert data to torch tensor
        self.data = torch.tensor(df.values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        #data_item = self.data[index]
        # Return a dictionary with key 'data' and the data as its value
        item_dict = dotdict()
        item_dict['data'] = self.data[index]
        return  item_dict

# Create an instance of MyDataset with the file path of your CSV file
real_data = MyDataset(real_data)
#real_data=torch.Tensor(real_data.values)

# Splitting into train and test datasets
train_size = int(0.8 * len(real_data))
test_size = len(real_data) - train_size
train_dataset, test_dataset = random_split(real_data, [train_size, test_size])

# Create data loaders for train and test datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [9]:
out = next(iter(train_loader))

out.data.shape

torch.Size([32, 21])

In [19]:
#using encoder and decoder to fit in VAE model
#(source:https://pythae.readthedocs.io/en/latest/models/autoencoders/vae.html)
#(sample:https://colab.research.google.com/github/clementchadebec/benchmark_VAE/blob/main/examples/notebooks/models_training/vae_training.ipynb)
#(source code:https://pythae.readthedocs.io/en/latest/_modules/pythae/models/vae/vae_model.html#VAE)

from pythae.models import VAE, VAEConfig
from pythae.trainers import BaseTrainerConfig
from pythae.pipelines.training import TrainingPipeline


config = BaseTrainerConfig(
    output_dir='my_model',
    learning_rate=2e-4,
    per_device_train_batch_size=500,
    per_device_eval_batch_size=500,
    num_epochs=30, # we can change this to train the model a bit more
)


model_config = VAEConfig(
    input_dim=(21,), #I also have used '(51,10000)' but the error still occurs
                     # Also I found here https://github.com/clementchadebec/benchmark_VAE/issues/81
    latent_dim=64
)

model = VAE(
    model_config=model_config,
    encoder=None, 
    decoder= None
)

In [20]:
#building the training pipeline
#source:https://pythae.readthedocs.io/en/latest/pipelines/pythae.pipelines.html
pipeline = TrainingPipeline(
    training_config=config,
    model=model
)

In [21]:
#launcing pipeline
#source code: https://pythae.readthedocs.io/en/latest/_modules/pythae/pipelines/training.html#TrainingPipeline
pipeline(
    train_data=train_dataset,
    eval_data=test_dataset
)

Checking train dataset...
Checking eval dataset...
Using Base Trainer

Model passed sanity check !
Ready for training.

Created my_model\VAE_training_2023-05-16_13-43-07. 
Training config, checkpoints and final model will be saved here.

Training params:
 - max_epochs: 30
 - per_device_train_batch_size: 500
 - per_device_eval_batch_size: 500
 - checkpoint saving every: None
Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0002
    maximize: False
    weight_decay: 0
)
Scheduler: None

Successfully launched training !



Training of epoch 1/30:   0%|          | 0/2 [00:00<?, ?batch/s]

ArithmeticError: NaN detected in train loss

In [18]:
train_loader

In [19]:
test_dataset

In [20]:
for i in enumerate(train_loader):
    print(i)

(0, tensor([[5.4600e+02, 2.4000e+01, 2.0220e+03,  ..., 0.0000e+00, 1.0000e+00,
         1.0000e+00],
        [1.6300e+02, 1.0000e+01, 7.3080e+03,  ..., 0.0000e+00, 1.0000e+00,
         1.0000e+00],
        [8.1000e+01, 1.5000e+01, 1.2130e+03,  ..., 0.0000e+00, 1.0000e+00,
         1.0000e+00],
        ...,
        [3.1600e+02, 1.2000e+01, 7.0800e+02,  ..., 1.0000e+00, 0.0000e+00,
         1.0000e+00],
        [9.9900e+02, 4.5000e+01, 4.5760e+03,  ..., 0.0000e+00, 0.0000e+00,
         1.0000e+00],
        [5.6700e+02, 2.4000e+01, 2.5780e+03,  ..., 0.0000e+00, 0.0000e+00,
         1.0000e+00]]))
(1, tensor([[1.7600e+02, 1.2000e+01, 2.5770e+03,  ..., 0.0000e+00, 0.0000e+00,
         1.0000e+00],
        [8.3100e+02, 1.8000e+01, 1.2160e+03,  ..., 0.0000e+00, 1.0000e+00,
         1.0000e+00],
        [9.0800e+02, 1.5000e+01, 3.5940e+03,  ..., 1.0000e+00, 0.0000e+00,
         1.0000e+00],
        ...,
        [9.1700e+02, 6.0000e+00, 1.4896e+04,  ..., 0.0000e+00, 1.0000e+00,
         1.0000e